<a href="https://colab.research.google.com/github/JamersonCarlos/Algoritmo-A-Google-Colab/blob/main/Algoritmo_A_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from re import A
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2
from google.colab.patches import cv2_imshow
import time

def realimentandoImagem():
   #---Inicio da Realimentação
  img = np.ones((50,50, 3), dtype="uint8") * 255
  img[start_point[0], start_point[1]] = start
  img[end_point[0], end_point[1]] = end_color

  for i in range(49, 0 , -1):
    img[i, 17] = obstacle

  for i in range(17,5,-1):
    img[40,i] = obstacle

  #Pintando as fronteiras exploradas
  for cordenadas_caminho in list_open_aux:
    img[cordenadas_caminho[0], cordenadas_caminho[1]] = expanded

  #---Fim da Realimentação
  return img

#Convertendo uma lista de imagens para um vídeo  
def imagens_para_video(lista_de_imagens, nome_video, fps):
    reshape = [cv2.resize(e,dsize=(500,500),interpolation=cv2.INTER_NEAREST_EXACT) for e in lista_de_imagens]
    tamanho = reshape[0].shape[1], reshape[0].shape[0]
    video = cv2.VideoWriter(nome_video, cv2.VideoWriter_fourcc(*'mp4v'), fps, tamanho)

    for imagem in reshape:
        video.write(imagem)
    cv2.destroyAllWindows()
    video.release()

#Encontrando os vizinhos
def definindoFronteira(img, atual_point):
  list_vizinhos = []

  #UP
  if((atual_point[0] - 1) >= 0 and img[atual_point[0] - 1, atual_point[1]][1] != 0):
    list_vizinhos.append([atual_point[0] - 1, atual_point[1]])

  #RIGHT
  if((atual_point[1] + 1) <= 49 and img[atual_point[0], atual_point[1] + 1][1] != 0):
    list_vizinhos.append([atual_point[0], atual_point[1] + 1])

  #DOWN
  if((atual_point[0] + 1) <= 49 and img[atual_point[0] + 1, atual_point[1]][1] != 0):
    list_vizinhos.append([atual_point[0] + 1, atual_point[1]])

  #LEFT
  if((atual_point[1] - 1) >= 0 and img[atual_point[0], atual_point[1] - 1][1] != 0):
    list_vizinhos.append([atual_point[0], atual_point[1] - 1])

  return list_vizinhos

#Calcula custo de um determinado ponto para o destino final
def custo_end(point):
  return abs((point[0] - end_point[0])) + abs((point[1] - end_point[1]))

#Calcula custo de um determinado ponto para o inicio. 
def custo_start(point):
  return abs(point[0] - start_point[0] + abs(point[1] - start_point[1]))


#Iniciando Variáveis
start_point = [15,10]
end_point = [35,25]
atual_point = start_point
caminho = [atual_point]
list_imagens = []

#cores
expanded = (56, 24, 76)
start = (255, 0, 0)
end_color = (0, 5, 255)
current = (206, 240, 157)
obstacle = (0,0,0)

#Variáveis do Algoritmo
list_open = [{
    'location': start_point,
    'custo_start': 0,
    'custo_end': 0,
    'custo_total': 0,
    'location_father': start_point
}]
list_open_aux = []
list_close_aux = []
list_close = []

#Execução
end = False
blocked = False
while end != True:

  img = realimentandoImagem()

  #Vizinhos do ponto atual
  vizinhos = definindoFronteira(img, list_open[0]['location'])

  #Obtendo informações dos vizinhos
  for vizinho in vizinhos:
    if (not vizinho in list_close_aux and not vizinho in list_open_aux):
      img[vizinho[0], vizinho[1]] = expanded
      list_open_aux.append(vizinho)
      list_open.append({
          'location': vizinho,
          'custo_start': custo_start(vizinho),
          'custo_end': custo_end(vizinho),
          'custo_total': (custo_start(vizinho) + custo_end(vizinho)),
          'location_father': list_open[0]['location']
      })

  #Incluindo pontos verificados na lista fechada
  item_remove = list_open.pop(0)
  list_close_aux.append(item_remove['location'])
  list_close.append(item_remove)

  # #Ordenando por custo
  list_open = sorted(list_open, key=lambda x: x['custo_total'])

  if (list_open != []):
    atual_point = list_open[0]['location']
    img[atual_point[0], atual_point[1]] = current
    list_imagens.append(img)
    if (atual_point == end_point):
      item_remove = list_open.pop(0)
      list_close_aux.append(item_remove['location'])
      list_close.append(item_remove)
      end = True
  else:
    #Indica que não há caminho até o ponto incial. 
    blocked = True
    end = True



#Criando Video Interativo para mostrar caminho
searchVariable = list_close[-1]
caminho_completo = []

def caminhoCompleto():
  global searchVariable
  global caminho_completo
  for i in range(len(list_close) - 1, 0, -1):
    if(list_close[i]['location'] == searchVariable['location_father']):
        searchVariable = list_close[i]
        caminho_completo.append(searchVariable)

if (not blocked):
  caminhoCompleto()
  for i in range(50):
    img = realimentandoImagem()
    for i in caminho_completo:
      img[i['location'][0], i['location'][1]] = current

    img[end_point[0], end_point[1]] = end_color
    list_imagens.append(img)

nome_video = 'video1.mp4'
fps = 10

imagens_para_video(list_imagens, nome_video, fps)